# Lecture #13: Model Selection and Parallel Tempering
## AM 207: Advanced Scientific Computing
### Stochastic Methods for Data Analysis, Inference and Optimization
### Fall, 2019

<img src="fig/logos.jpg" style="height:150px;">

In [2]:
### Import basic libraries
from autograd import numpy as np
from autograd import grad
import numpy
import scipy as sp
import pandas as pd
import sklearn as sk
from sklearn.datasets.samples_generator import make_blobs
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
from IPython.display import HTML
from IPython.display import YouTubeVideo
%matplotlib inline

## Administrative Matters

1. **Attendance Quiz:** 
2. **Projects:** Project mentors will be announced on Canvas (there will be one TF assigned for each paper). Set up times to meet with your project mentor and instrctur (together or separately) - see Check Point #2.

## Outline
1. Model Selection for Hierachical Generalized Linear Models
2. MCMC Diagnostics
3. Parallel Tempering

# Model Selection for Hierachical Generalized Linear Models

## How Many Covariates to Include?

Now we can inject covariates into any distribution $Y^{(n)} \sim p(Y | \theta)$, by making $\theta$ a function of $\mathbf{X}$. Doing so allows us to explain the distribution of the outcome $Y$ based on explanatory factors $\mathbf{X}$. 

In fact, **the more covaraiates you use, the better your model will fit the data**. 

But interpreting the model becomes problemmatic when the covariates are not independent of each other.

## The Dangers of Model Interpretation

For example, suppose that you are modeling kidney cancer using a logistic regression model with $\mathbf{X} = \{\text{age}, \text{smoker}\}$. Suppose that the maximum likelihood model you found is

$$
\mathrm{Prob}[Y = 1| \mathbf{X}, \mathbf{w}] = \mathrm{sigm}(-5 * \text{age} - 0.2 * \text{smoker} - 0.5)
$$

What happens if you included more covariates: $\mathbf{X} = \{\text{age}, \text{video games (hr)}, \text{smoker}\}$? If the new covariates are correlated with existing ones, e.g. 

$$\text{video games (hr)} = 0.5 * \text{age},$$ 

then the model weights can change drastically:

$$
\mathrm{Prob}[Y = 1| \mathbf{X}, \mathbf{w}] = \mathrm{sigm}(0.5 * \text{age} - 11 * \text{video games (hr)} - 0.2 * \text{smoker} - 0.5)
$$

Note, both models will fit the observed data equally well!

## Model Selection through Cross-Validation
If one of these models captures the true relationship between the covariates and the risk of kidney cancer, then it will ***generalize*** (fit new data) well.

\begin{align}
(\text{Model 1})\quad \mathrm{Prob}[Y = 1| \mathbf{X}, \mathbf{w}] &= \mathrm{sigm}(-5 * \text{age} - 0.2 * \text{smoker} - 0.5)\\
(\text{Model 2})\quad \mathrm{Prob}[Y = 1| \mathbf{X}, \mathbf{w}] &= \mathrm{sigm}(0.5 * \text{age} - 11 * \text{games} - 0.2 * \text{smoker} - 0.5)
\end{align}

In other words, if one of these models is capturing spurious connections between the covariates and the outcome, then it should fail on new data where this connection doesn't hold (e.g. when we collect data on older individuals who play a lot of video games).

One way to simulate the model's performance on new data is to randomnly hold-out different parts of the observed data during inference (we train on the remaining data) and use the held-out data to test the performance of the model. We select the model that has the best performance, on average, on held-out data. This is called ***cross-validation***. 

Note: in cross validation you are re-training your model multiple times. This can be prohibitively expensive!

## Model Selection for Maximum Likelihood Models

An alternative to model selection via cross-validation is to directly approximate the model's performance on new data. We evaluate predictive accuracy using the log-likelihood of the observed data:
$$
D(y|\mathbf{w}_{\text{MLE}}) = -2\sum_{n=1}^N\log p\left(Y^{(n)}|\mathbf{X}^{(n)}, \mathbf{w}_{\text{MLE}}\right)
$$
We know that this is an overestimate of the log-likelihood of new data under the model. So a correction must be made. This correction is typically the complexity of the model, i.e. the dimension of $\mathbf{w}$:

1. (**Akaike's Information Criterion**) $$\text{AIC} = D(y|\mathbf{w}_{\text{MLE}}) + 2\,\mathrm{dim}(\mathbf{w}_{\text{MLE}})$$

2. (**Bayesian Information Criterion**) $$\text{BIC} = D(y|\mathbf{w}_{\text{MLE}}) + \log(N)\,\mathrm{dim}(\mathbf{w}_{\text{MLE}})$$

The smaller the number the better.

The validity of each criterion is argued asymptotically, assuming a number of specific modeling conditions -- in practice, use with caution!

## Model Selection for Bayesian Models

For Bayesian models we evaluate the predictive accuracy using the (point-wise) log posterior predictive likelihood (lppd) of the observed data:

$$
\mathrm{lppd} = \sum_{n=1}^N\log \int p\left(Y^{(n)}|\mathbf{X}^{(n)}, \mathbf{w}\right) p(\mathbf{w} | \text{Data}) d\mathbf{w} = \sum_{n=1}^N\log\mathbb{E}_{\mathbf{w} \sim p(\mathbf{w} | \text{Data})}\left[p\left(Y^{(n)}|\mathbf{X}^{(n)}, \mathbf{w}\right) \right].
$$

Again, this is an overestimate of the log-likelihood of new data under the posterior. So a correction must be made. In this case, the correction is the (point-wise) variance of the log posterior predictive likelihood

$$
p_{\text{WAIC}} = \sum_{n=1}^N\log\mathrm{Var}_{\mathbf{w} \sim p(\mathbf{w} | \text{Data})}\left[p\left(Y^{(n)}|\mathbf{X}^{(n)}, \mathbf{w}\right) \right].
$$

The ***Watanabe-Akaike information criterion (WAIC)*** is defined to be

$$
\mathrm{WAIC} = -2\, \mathrm{lppd} + 2\, p_{\text{WAIC}}.
$$

One may interpret $p_{\text{WAIC}}$ as an approximation of the effective number of parameters in the model (in a complex hierarchical model the "number of parameters" is not obvious to quantify).

# MCMC Diagnostics

## Hamiltonian Monte Carlo (HMC)

We use the ***Gibbs distribution*** to transform between probability density functions and energy functions
$$
U(q) = - \log \pi(q), \quad \pi(q) = \frac{1}{Z}\exp\left\{ \frac{-U(q)}{T}\right\},\; T=1
$$
This allows us to use gradient information when we sample from $\pi(q)$.

<img src="fig/hamiltonian.jpg" style="height:300px;">

## What's Important About HMC?

1. **Question:** HMC seems complicated, do I really need to use it?

  **Answer:** Yes. For practically any model of interest (non-conjugate, hierarchical, latent variable) HMC is the  least complicated sampler you can use in order to perform reliable Bayesian inference.<br><br>

2. **Question:** Ok, but can I treat the theory as a black-box, i.e. can I just press some `model.sample()` button?

  **Answer:** No. HMC (just like all samplers) must be tuned. That is, for many models and datasets, `model.sample()` will not have good performance. You need the theory to tell you which design choices need to be adjusted and in what way.<br><br>
  
3. **Question:** But I don't need to implement it since `pymc3` has already done so, right?

  **Answer:** You'll need to implement HMC. This is because `pymc3` is not going to scale well for Bayesian inference for models with neural network likelihoods and large datasets.

## HMC for Multimodal Distributions

In [3]:
HTML("""<video height="440" controls><source src="fig/hmc_multimodal.mov" type="video/mp4"></video>""")

## Signs of Maybe Convergence?

Look for:
1. Large segments of the chain should have give similar statistics (mean, variance etc)
2. Low correlations within states of the chain
3. "Reasonably high" acceptance rate of proposed steps
4. Multiple chains initialized from different initial points give similar results

Best practics:
1. Always run multiple chains initialized from very different random starting points
2. Always run your chains for as long as you can then burn and thin
3. Always check all relevant convergence diagnostics
4. Never be too certain: remember that there is no "proof" of convergence for finite chains!
5. Keep reading about best practice!

## Visual Diagnostics: Traceplots of Multiple Chains

<img src="fig/multichain.jpg" style="height:400px;">

## Autocorrelation: the "Effective" Sample Size

We quantify how much the samples in the chain are correlated with the samples obtained $k$-steps later (**the $k$-th lag**). The ***autocorrelation*** $\rho_k$ is defined as

$$
\rho_k = \frac{\sum_{n=1}^{N-k}(x_n - \bar{x})(x_{n+k} - \bar{x})}{\sum_{n=1}^{N}(x_n - \bar{x})^2}
$$

We plot the autocorrelation for each $k = 1, \ldots, \frac{N}{2}$, and this ***autocorrelation plot*** tells us how much we to thin in order to obtain effectively independent samples. The autocorrelation plot gives us an idea of the ***effective sample size*** of the Markov chain.

## Visual Diagnostics: The Autocorrelation Plot

<img src="fig/autocorr.jpg" style="height:400px;">

## Quantitative Diagnostics

**Idea:** measure between-chain and within-chain variability of a quantity of interest – if the chains have converged, these measures will be similar; otherwise, the between-chain variability will be larger.

1. ***Gelman & Rubin***: quantity $\widehat{R}_{\text{GR}} =  \frac{B}{W}$, which compares $B$ the empirical variance of all the chains pooled and $W$ the average emprical variance within each chain. If $\widehat{R}_{\text{GR}}$ is large then the chains are very different (not converged). If $\widehat{R}_{\text{GR}} = 1$ is ideal but in practice we accept $\widehat{R}_{\text{GR}} < 1.05$.<br><br>

2. ***Geweke***: takes two nonoverlapping parts (usually the first 0.1 and last 0.5 proportions) of the Markov chain and compares the means of both parts, using a difference of means test to see if the two parts of the chain are from the same distribution (the test statistic is a standard Z-score with the standard errors adjusted for autocorrelation).

# Parallel Tempering

## Multimodal Posteriors

But when are posteriors multimodal? Often, the posterior can be multimodal when the likelihood is ***non-identifiable***, i.e. there are multiple sets of model parameters that can explain the data equally well.

For example, the observed data likelihood of a Gaussian mixture model with 2 univariate components is: 

$$
p(y|\mu, \sigma^2, \pi) = \pi_1 \mathcal{N}(y; \mu_1, \sigma^2_1) + \pi_2 \mathcal{N}(y; \mu_2, \sigma^2_2)
$$

But, given an observation $y$, there are multiple sets of model parameters $\mu, \sigma^2, \pi$ that will fit the data:

$$
0.1 \mathcal{N}(y; 1, 0.5) + 0.9 \mathcal{N}(y; -2, 1) = 0.9 \mathcal{N}(y; -2, 1) + 0.1 \mathcal{N}(y; 1, 0.5)
$$

That is, we can label the components however we want without changing the fit. 

When we fit a Bayesian GMM, the posterior will contain multiple modes, one for each way of labeling the components. 

**Note:** There are more non-trivial ways in which the likelihood of a GMM can be non-identifiable and, hence, its posterior multimodal! 

## The Effect of Temperature

Why is it hard for samplers to visit multiple modes in a target distribution? 

MCMC samplers can only propose locally, so moving from one mode to another requires traveling through regions that are very unlikely under the target distribution. 

In HMC terms, moving from one mode to another requires climbing a big hill -- this is called an ***energy barrier***. From simulated annealing we know that range of movement of a sampler of a Gibbs distribution is enhanced when we increase the temperature term:

$$
\pi(q) = \frac{1}{Z}\exp\left\{ -\frac{-\log\pi(q)}{T}\right\} 
$$

Another way to say this is that when temperature is high, the poential energy landscape $\frac{-\log\pi(q)}{T}$ is flat, hence easier to explore.

But we can't simply set $T > 1$! Doing so means that we will not be sampling from the target $\pi(q)$ (i.e. the above equation will not hold).

## The Idea of Parallel Tempering

Using one MCMC chain with $T>1$ will produce incorrect samples. What if we use multiple chains: one for $T=1$ (which will produce samples from $\pi(q)$) and other chains with $T>1$, and we allow the chains to exchange samples once in a while?

<img src="fig/tempering.jpg" style="height:300px;">

## Parallel Tempering with HMC

We set an increasing sequence of temperatures $T_0=1, \ldots, T_M >1$. For each temperature, denote the corresponding Gibbs distribution and potential energy function as follows:
$$
\pi_m(q) = \frac{1}{Z}\exp\left\{ -U_m(q)\right\}, \; U_m(q) = \frac{-\log\pi(q)}{T_m}
$$
We denote the potential energy of $\pi(q)$ as $U(q) = -\log\pi(q)$.

**Parallel Tempering HMC**

**1.** initialize $M$ number of HMC samplers: each using the same kinetic energy function, the $m$-th sampler using the potential energy function $U_m(q)$.

**2.**  alternate between:<br>
$\quad$**a.**  sample $S$ samples from each chain independently<br>
$\quad$**b.**  at the $S$-sample, the sample $q^{S}_{m+1}$ from chain $m + 1$ is exchanged with the sample $q^{S}_{m}$ from chain $m$ with the probability: 
  
  $$
  \alpha = \min\left\{1, \exp \left\{(1/T_{m} - 1/T_{m+1})(U(q^{S}_m) - U(q^{S}_{m+1}))\right\}\right\}
  $$
  
In the end, we keep the samples in the $0$-th chain.

Can you show that parallel tempering is detailed balanced? What about irreducible and aperiodic?